# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [41]:
# import libraries

# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np
import joblib

nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
# load data from database
engine = create_engine('sqlite:///test_db.db')
df = pd.read_sql_table('test_table2', engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
category_names = Y.columns.tolist()
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [31]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punc_table = str.maketrans('', '', string.punctuation)
def tokenize(text):
    # normalize case and remove punctuation
    text = text.translate(remove_punc_table).lower()
    
    # tokenize text
    tokens = nltk.word_tokenize(text)
    
    # lemmatize and remove stop words
    return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

forest_clf = RandomForestClassifier(n_estimators=5)

pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('forest', MultiOutputClassifier(forest_clf))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
from sklearn.metrics import classification_report
Y_pred = pipeline.predict(X_test)
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.60      0.41      0.49      2017
          1       0.83      0.92      0.87      6551
          2       0.41      0.13      0.19        71

avg / total       0.77      0.79      0.77      8639

request
             precision    recall  f1-score   support

          0       0.89      0.96      0.92      7156
          1       0.68      0.45      0.54      1483

avg / total       0.86      0.87      0.86      8639

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8600
          1       0.00      0.00      0.00        39

avg / total       0.99      1.00      0.99      8639

aid_related
             precision    recall  f1-score   support

          0       0.76      0.80      0.78      5070
          1       0.69      0.63      0.66      3569

avg / total       0.73      0.73      0.73      8639

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
from sklearn.model_selection import GridSearchCV


parameters = {

    'forest__estimator__n_estimators': [2,4],
    'forest__estimator__min_samples_split': [20]
}

cv = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1)
cv.fit(X_train, Y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'forest__estimator__n_estimators': [2, 4], 'forest__estimator__min_samples_split': [20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
from pprint import pprint
pprint(cv.best_estimator_.steps)

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f048e213a60>, use_idf=True,
        vocabulary=None)),
 ('forest',
  MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=4, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]


In [37]:
Y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
1968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17678,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1087,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7172,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5456,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21463,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9296,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23206,1,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 8. Export your model as a pickle file

In [42]:
joblib.dump(cv.best_estimator_, 'test_model.pkl')


['test_model.pkl']